In [24]:
import numpy as np

## Viscosity Functions
From "Fundamentals of Machine Component Design" Section 12, pp 633-635
Reference TBL12-1

Note: 

(F-32)*5/9 = C

6.89 * microreyn = milli-PascalSec  --> 6.89 * microreyn /1000 = PascalSec --> 6.89 * 1000 reyn = PascalSec

In [44]:
# Idek what the fuck that scaling is on the plot. 
# Its like exponential scaling, which I have never even seen before.
# It shows empiral formulas for the basic SAE oils tho
def Visc_SAE_Seireg(Temp, Grade, Metric = False):
    if (Metric):
        Temp = (Temp * 9/5) + 32 # C to F
    match Grade:
        case 10:
            m0 = 0.0158E-6
            b = 1157.5
        case 20:
            m0 = 0.0136E-6
            b = 1271.6
        case 30:
            m0 = 0.0141E-6
            b = 1360.0
        case 40:
            m0 = 0.0121E-6
            b = 1474.4
        case 50:
            m0 = 0.0170E-6
            b = 1509.6
        case 60:
            m0 = 0.0187E-6
            b = 1564.0
    if (not Metric):
        return m0 * np.e**(b / (Temp + 95))    
    else:
        return m0 * np.e**(b / (Temp + 95)) * 6.89 * 1000 # convert back to Metric

# Exponential Fit of the Below Data Source, as found in the GDrive
# https://wiki.anton-paar.com/us-en/engine-oil/
def Visc_SAE_Inter(Temp, Grade, Metric = False):
    match Grade:
        case "10W-40":
            if (Metric): mu = 6.61 * np.e**(-0.0484 * Temp)
            else: mu = 2.27E-3 * np.e**(-0.0269 * Temp)
        case "30":
            if (Metric): mu = 18 * np.e**(-0.0708 * Temp)
            else: mu = 9.18E-3 * np.e**(-0.0393 * Temp)
    return mu

            

In [53]:
# Sommerfeld Calculation for Naturally Lubricated Journal Bearings
def Sommerfeld(mu, r, c, N, p, echo = False):
    S = (float(r)/float(c))**2 * float(mu) * float(N) / float(p) 
    if(echo): print("Sommerfeld #: " + str(S))
    return S 

# Calculation of temperature change via the Temperature Factor (see below)
def TempDelta(Tval, p, iter = 0):
    delT = Tval * p / 9.7 # for Imperial
    if iter != 0:
        print("Temperature Delta (" + str(iter) + "): " + str(delT))    
    return delT

# Calculation of Temperature Factor for limited L/D ratio
# TODO: Generalize for l/d ratios
def TempFactor(S, l, d, echo = False):
    if(abs(l - d) < .001):
        factT = .349109 + 6.00940 * S + .047467 * S**2
    elif(abs(l/d - .5) < .001):
        factT = .394552 + 6.392527 * S - .036013 * S**2
    elif(abs(l/d - .25) < .001):
        factT = .933828 + 6.437512 * S - .011048 * S**2
    else: 
        factT = NULL
    if (echo): print("Temp Factor: " + str(factT))
    return(factT)

# Interchange between Metric and Imperial. "Metric" is if the input is Celsius to begin with, and will convert to F
def ConvertTemp(Temp, Metric = False):
    if (Metric):
        return (Temp * 9/5) + 32
    else:
        return (Temp - 32) * 5/9


# EMAE370 Chapter 12: Journal Bearings

## Iterative Approach

In [57]:
Grd = 30 #SAE30

load_lbf = 10E3
rpm = 900
Tin_degF = 180

l_in = 4
r_in = 4

p_psi = load_lbf / (2*r_in * l_in)

c = r_in*2/1000

rps = 900 / 60

print("Pressure (psi): " + str(p_psi))

Pressure (psi): 312.5


In [73]:
# -- Natural Journal Function Definition -- 
# Grade = Oil Grade, as a String ("10W-40") for 
def NaturalJournal(Grade, Tout_guess, T_in, load, rpm, l, r, c, convergence = 0.1, Metric = False,):
    p = load / (2 * r * l)
    rps = rpm /60 

    Tavg_guess = (T_in + Tout_guess)/2

    # Determining if it needs to be parsed as interpolated or Seireg for calculating mu

    if(isinstance(Grade, str)): 
        try: 
            mu_guess = Visc_SAE_Inter(Temp = Tavg_guess, Grade = Grade, Metric = Metric)
            method = "Inter"
        except: print("Likely could not Parse Grade")
    if(isinstance(Grade, int)):
        try: 
            mu_guess = Visc_SAE_Seireg(Temp = Tavg_guess, Grade = Grade, Metric = Metric)
            method = "Seireg"
        except: print("Likely could not Parse Grade")
    
    print(type(mu_guess))
    print(type(r))
    print(type(c))
    print(type(rps))
    print(type(p))
    S_guess = Sommerfeld(mu = mu_guess, r = r, c = c, N = rps, p = p)
    delT_guess = TempDelta(Tval = TempFactor(S = S_guess, l = l, d = 2*r), p = p)

    error = convergence*1.1 # Throwaway to run while loop
    n = 0
    while error > convergence:
        
        # Keeping track of iterative values
        Tavg_prev = Tavg_guess
        mu_prev = mu_guess
        S_prev = S_guess
        delT_prev = delT_guess
        n = n+1

        Tout_guess = T_in + delT_guess
        Tavg_guess = T_in + delT_guess/2
        match method:
            case "Inter": mu_guess = Visc_SAE_Inter(Temp = Tavg_guess, Grade = Grade, Metric = Metric)
            case "Seireg": mu_guess = Visc_SAE_Seireg(Temp = Tavg_guess, Grade = Grade, Metric = Metric)
        S_guess = Sommerfeld(mu = mu_guess, r = r, c = c, N = rps, p = p)
        delT_guess = TempDelta(Tval = TempFactor(S = S_guess, l = l, d = 2*r), p = p)

        error = abs(delT_guess - delT_prev)

    print("Converged on Iteration:" + '\t' + str(n))
    print("Sommerfeld:" + '\t' + str(S_guess))
    print("Outlet Temperature:" + '\t' + str(T_in + delT_guess))
    return(S_guess, T_in + delT_guess)


In [74]:
NaturalJournal(Grade = Grd, Tout_guess = 205, T_in = Tin_degF, load = load_lbf, rpm = rpm, l = l_in, r = r_in, c = c)


<class 'float'>
<class 'int'>
<class 'float'>
<class 'float'>
<class 'float'>
Converged on Iteration:	2
Sommerfeld:	0.02051117870129257
Outlet Temperature:	196.9347652279757


(0.02051117870129257, 196.9347652279757)

In [64]:
thistuple = ("apple",)
print(type(thistuple))

<class 'tuple'>


In [43]:
mu = 1.5981621311834866e-06
r = 4
c = 0.008
N = 15
p = 312.5
(r/c)**2 * mu * N / p
#Sommerfeld(mu= 1.5981621311834866e-06, r = 4, c = .008, N = 15, p = 312.5)

0.01917794557420184

In [ ]:
# Guess Temperature
Tout_g = 205
Tavg_g = (Tin_degF + Tout_g)/2
print("Guess Film Average Temp, degF: " + str(Tavg_g))

In [ ]:
Tout = Tout_g

error = 1.1 # throwaway value
firstIterFlag = True
convergence_res = .1 # deg F
n = 1
while error > convergence_res:
    if firstIterFlag:   
        Tavg = (Tin_degF + Tout)/2
        mu = Visc_SAE_Seireg(Temp = Tavg, Grade = Grd, Metric = False)
        print("Viscosity: " + str(mu))
        S = Sommerfeld(mu = mu, r = r_in, c = c, N = rps, p = p_psi)
        delT = TempDelta(Tval = TempFactor(S = S, l = l_in, d = 2*r_in), p = p_psi, iter = n)
        firstIterFlag = False
    
    n = n+1
    # Storing for comparison to exit while loop
    Tavg_prev = Tavg
    mu_prev = mu 
    S_prev = S 
    delT_prev = delT 

    Tout = Tin_degF + delT 
    Tavg = Tin_degF + delT/2
    mu = Visc_SAE_Seireg(Temp = Tavg, Grade = Grd, Metric = False)
    S = Sommerfeld(mu = mu, r = r_in, c = c, N = rps, p = p_psi)
    delT = TempDelta(Tval = TempFactor(S = S, l = l_in, d = 2*r_in), p = p_psi, iter = n)

    error = abs(delT - delT_prev) 
    if (error <= convergence_res):
        print("S ~ last iteration Sommerfeld: " + str(S) + '\n' + "Tout ~ last iteration Temp: " + str(Tin_degF + delT))
        output = (S, Tin_degF + delT)



## Manual Approach

In [ ]:
## SAE 30

load_lbf = 10E3
rpm = 900
Tin_degF = 180

l_in = 4
r_in = 4

# --- #

p_psi = load_lbf / (2*r_in * l_in)

c = .008

rps = 900 / 60

print("Pressure (psi): " + str(p_psi))

In [ ]:
Tout_1 = 205
Tavg_1 = (Tin_degF + Tout_1)/2
print("Guess Film Average Temp, degF: " + str(Tavg_1))

In [ ]:
mu_1 = 1.4E-6

In [ ]:
S_1 = Sommerfeld(mu = mu_1, r = r_in, c = c, N = rps, p = p_psi)

In [ ]:
delT_1 = TempDelta(Tval = TempFactor(S = S_1, l = l_in, d = 2*r_in), p = p_psi, iter = 1)

T_2 = Tin_degF + delT_1
print(T_2)

In [ ]:
Tavg_2 =  Tin_degF + delT_1/2
print("Guess Film Average Temp, degF: " + str(Tavg_2))

In [ ]:
mu_2 = 1.8E-6
S_2 = Sommerfeld(mu = mu_2, r = r_in, c = c, N = rps, p = p_psi)
delT_2 = TempDelta(Tval = TempFactor(S = S_2, l = l_in, d = 2*r_in), p = p_psi, iter = 2)
T_3 = Tin_degF + delT_2
print(T_3)

In [ ]:
Tavg_3 =  Tin_degF + delT_2/2
print("Guess Film Average Temp, degF: " + str(Tavg_3))

In [ ]:
mu_3 = 1.8E-6
S_3 = Sommerfeld(mu = mu_3, r = r_in, c = c, N = rps, p = p_psi)
delT_3 = TempDelta(Tval = TempFactor(S = S_3, l = l_in, d = 2*r_in), p = p_psi, iter = 3)
T_4 = Tin_degF + delT_3

In [ ]:
Tout = Tin_degF + delT_3
print(Tout)

In [ ]:
h_0 = c * .8
print("h_0: " + str(h_0))
f = c/r_in * 1.4
print("f: " + str(f))
Q = 5.85 * r_in * c * rps * l_in
print("Qfull: " + str(Q))
Qs = .94 * Q 
print("Side Flow Qs: " + str(Qs))
